## TODO



*   Do without Global

*   Reduce the time in Initial for loop (seqs) 
*   Write the dataframe to file


*   What to do when start_index + frag_len > total_seq_len


*   Introduce a unique column for Dataframe


*   Change getSequence function to return a pd.Series









##Install **Packages** and mount **Drive**

In [ ]:
!pip install biopython
#!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
# torch.cuda.is_available()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Imports

In [ ]:
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
from Bio import SeqIO
from datetime import datetime
from numpy.random import randint
# import torch

##**Constants** and **Values**

Constants

In [ ]:
dnaStringPattern = "^[ACGT]+$"
stringExtractionPattern = '[ACTG]+'
maximumFragments = float("inf")

User Imputs using CMD

In [ ]:
inputFile = "/content/drive/Shareddrives/FYP/Databases/PLSdb-plasmid.fasta"
miniInputFile = "/content/drive/Shareddrives/FYP/Databases/PLSdb-plasmid-1000.fasta"
minLength = 1000
verbose = True
lengths = [1000,10000,100000,500000]
coverage = 5
fragmentRatio = 0.85

inputFile = miniInputFile

Column Names

In [ ]:
contigId = "contig_id"
contigLength = "contig_length"
contigName = "contig_name"
contigSequence = "contig_sequence"
regexLists = "regex_list"
regexListCount = 'regex_list_count'

Global Variables

In [ ]:
global seqCount
seqCount = 0

global df

global totalLength

##**Supporting Methods**

In [ ]:
def countSequencesInFile():
  global seqCount
  seqCount +=1
  return True

In [ ]:
def PrintToConsole(mes, arg1 = "", arg2 = "", arg3 = ""):
  if(verbose):
    print(mes, arg1, arg2, arg3)

In [ ]:
def removeUnwantedSeqs(row):
  global df, totalLength
  seqs = np.array(row[regexLists])
  newFrame = pd.DataFrame([
      {contigId: i + totalLength, contigName: row[contigName] + "_" + str(i), contigLength: len(rec), contigSequence: str(rec[0:])}
      for i, rec in enumerate(seqs)
  ])
  frames = [df, newFrame]
  df = pd.concat(frames)
  totalLength = len(df)

In [ ]:
def getSequence(sequence, length, l):
  if length < l:
    startIndex = 0
    endIndex = length
  else:
    #TOASK:// In Plassclass they assume circularity, We cannot do this because we divide the sequence
    startIndex = randint(0, length - l)
    endIndex = startIndex + l
  return sequence[startIndex : endIndex]

##**Implementation**

In [ ]:
# TODO:// When there is a error. Split the sequence left and right

def parse_seqs(inputfile, min_length = 1000):

    PrintToConsole("Importing sequences")
    startTime = datetime.now()

    initialSequences = SeqIO.parse(inputfile, 'fasta')

    PrintToConsole("Reading Time \t", datetime.now() - startTime)
    startTime = datetime.now()
    
    #TODO:// Can create as a numpy array
    seqs = [rec for rec in initialSequences] # NOTE:// np.str got 53 seconds, normal methods 41

    PrintToConsole("Time to initial for loop ",datetime.now() - startTime)
    startTime = datetime.now()

    PrintToConsole("Imported ", len(seqs), " sequences")

    contig_info = pd.DataFrame([
        {contigId: i, contigName: rec.id, contigLength: len(rec.seq), contigSequence: str(rec.seq[0:])}
        for i, rec in enumerate(seqs)
    ])

    PrintToConsole(" Time to create the dataframe ", datetime.now() - startTime)
    startTime = datetime.now()

    PrintToConsole("Total length sequences ", len(contig_info))
    
    return seqs, contig_info

In [ ]:
seqs, contig_info = parse_seqs(inputFile, min_length=minLength)

Importing sequences   
Reading Time 	 0:00:00.010763  
Time to initial for loop  0:00:00.666158  
Imported  1000  sequences 
 Time to create the dataframe  0:00:00.062492  
Total length sequences  1000  


In [ ]:
df = contig_info.copy()
df
totalLength = len(df)

In [ ]:
contig_copy = contig_info.copy()

startTime = datetime.now()
contig_copy[regexLists] = contig_info.apply(lambda row: re.findall(stringExtractionPattern,row[contigSequence]), axis = 1)
print(datetime.now() - startTime)

contig_copy[regexListCount] = contig_copy[regexLists].apply(lambda row: len(row))

df = contig_copy.loc[contig_copy[regexLists].apply(lambda row: len(row) == 1)]

contig_copy_with_lengths = contig_copy.loc[contig_copy[regexLists].apply(lambda row: len(row) > 1)]
contig_copy_with_lengths

0:00:00.274333


,contig_id,contig_name,contig_length,contig_sequence,regex_list,regex_list_count
21,21,NZ_AF304384.1,8135,GTCGACAGTCCATGCTTCGAGTTGTTGCTGGAATTGTTGCGCTTGA...,[GTCGACAGTCCATGCTTCGAGTTGTTGCTGGAATTGTTGCGCTTG...,8
65,65,NZ_CP013044.1,42209,TATATCAAAGTTGAAATAATGGGGATAAAATAAAAGTCTAAAAAGA...,[TATATCAAAGTTGAAATAATGGGGATAAAATAAAAGTCTAAAAAG...,2
69,69,NC_022355.1,55127,TTGACGGAAAAAGGTAAAAAAAACTCATTGATAAGATTAGCGGTCA...,[TTGACGGAAAAAGGTAAAAAAAACTCATTGATAAGATTAGCGGTC...,2
134,134,NZ_CP024891.1,90897,CGGTGAACGTCCCCGCCGCCCGTGCCTGGGACAGCGCGGTCGACCC...,[CGGTGAACGTCCCCGCCGCCCGTGCCTGGGACAGCGCGGTCGACC...,2
491,491,MF554637.1,116894,GTGACCGGCGAGCAGAGCCTGCGCATGCACATGTCTCTGCATCCTT...,[GTGACCGGCGAGCAGAGCCTGCGCATGCACATGTCTCTGCATCCT...,3
512,512,NZ_KY689632.1,232345,GATTGATAGCGACGTTATGTGAATATTGAAATGAGCATTCCAAAGA...,[GATTGATAGCGACGTTATGTGAATATTGAAATGAGCATTCCAAAG...,6
527,527,NZ_KX858825.1,89970,GTGAATGACAATCAGATTTCCGTTCACTGGTCAGATCTACCGAAAG...,[GTGAATGACAATCAGATTTCCGTTCACTGGTCAGATCTACCGAAA...,3
549,549,NC_018265.1,199075,TTTTTATAGATAGAAAAAGAGATACAGCAACCTGCTGTATCTCCTT...,[TTTTTATAGATAGAAAAAGAGATACAGCAACCTGCTGTATCTCCT...,2
714,714,NZ_KY271415.1,55680,CAGTTGATTGGGCGTAATGGCTGTTGTGCAGCCAGCTCCTGACAGT...,[CAGTTGATTGGGCGTAATGGCTGTTGTGCAGCCAGCTCCTGACAG...,41
746,746,NZ_KY515226.1,105994,CTGGCTGTATAAATGCCATCCCTGGTCGTCGTCCAGTTCAATTACA...,[CTGGCTGTATAAATGCCATCCCTGGTCGTCGTCCAGTTCAATTAC...,11


In [ ]:
expectedLength = len(contig_info) - len(contig_copy_with_lengths) + np.sum(contig_copy_with_lengths[regexListCount])
expectedLength

1107

In [ ]:
startTime = datetime.now()
contig_copy_with_lengths.apply(lambda row: removeUnwantedSeqs(row), axis = 1)
print(datetime.now() - startTime)

0:00:00.079546


In [ ]:
print(totalLength)
df

1107


,contig_id,contig_name,contig_length,contig_sequence,regex_list,regex_list_count
0,0,NZ_CP045500.1,42856,GCCGCCGGAACTGCGCTGGCGCGAGTGGATGGGTCGTGTCGAAGCG...,[GCCGCCGGAACTGCGCTGGCGCGAGTGGATGGGTCGTGTCGAAGC...,1.0
1,1,NZ_CP045499.1,389300,TTGGAGAACACGGGGCTCTTTGAGAGGCGCCAGCCAGGGCCCGTTG...,[TTGGAGAACACGGGGCTCTTTGAGAGGCGCCAGCCAGGGCCCGTT...,1.0
2,2,NZ_CP045498.1,53938,TCGCATGCCGCATGCCGAGCTCGTCATGAGCGCGTTTGAGTTGCGC...,[TCGCATGCCGCATGCCGAGCTCGTCATGAGCGCGTTTGAGTTGCG...,1.0
3,3,NZ_CP045497.1,305508,ACGTACGGGCCCGCGACGAAATCGCCGACTTGCAATTCGATGCGGA...,[ACGTACGGGCCCGCGACGAAATCGCCGACTTGCAATTCGATGCGG...,1.0
4,4,NZ_MT560071.1,2516,TCAAGACGATAGTTACCGGATAAGGCGCAGCAGTCGGACTGAACGG...,[TCAAGACGATAGTTACCGGATAAGGCGCAGCAGTCGGACTGAACG...,1.0
...,...,...,...,...,...,...
2,1102,NZ_KX711708.1_2,575,TAAAAGAAAATGATAAAGAGTTTATTGGGCCACTTGCTATGATTAA...,NaN,NaN
3,1103,NZ_KX711708.1_3,3,TGT,NaN,NaN
4,1104,NZ_KX711708.1_4,27577,TTCACTGAACGTATTTGGCTGGCATTTATACAGGAGCACATGTGCA...,NaN,NaN
0,1105,NZ_KX774387.1_0,105067,AGCGGGCGGCCGGAAGGTGAATGCTAGGCATGATCTAACCCTCGGT...,NaN,NaN


In [ ]:
contig_info_df = df.copy()

startTime = datetime.now()
filtered_df = contig_info_df.loc[contig_info_df.apply(lambda row: row[contigLength] > minLength, axis = 1)]
PrintToConsole(" minimum length time ", datetime.now() - startTime)

PrintToConsole("Filtered length sequences ", len(filtered_df))
df = filtered_df[[contigId, contigLength, contigName, contigSequence]]
df

 minimum length time  0:00:00.011164  
Filtered length sequences  1036  


,contig_id,contig_length,contig_name,contig_sequence
0,0,42856,NZ_CP045500.1,GCCGCCGGAACTGCGCTGGCGCGAGTGGATGGGTCGTGTCGAAGCG...
1,1,389300,NZ_CP045499.1,TTGGAGAACACGGGGCTCTTTGAGAGGCGCCAGCCAGGGCCCGTTG...
2,2,53938,NZ_CP045498.1,TCGCATGCCGCATGCCGAGCTCGTCATGAGCGCGTTTGAGTTGCGC...
3,3,305508,NZ_CP045497.1,ACGTACGGGCCCGCGACGAAATCGCCGACTTGCAATTCGATGCGGA...
4,4,2516,NZ_MT560071.1,TCAAGACGATAGTTACCGGATAAGGCGCAGCAGTCGGACTGAACGG...
...,...,...,...,...
1,1099,2672,NZ_KX452393.1_1,GTAGGCAGTTCGCTCCAAGCTGGACTGTATGCACGAACCCCCCGTT...
0,1100,6723,NZ_KX711708.1_0,ATGAGAATGACGACAAATAAGACTTCCCTTTCTCGCTTAACAAAAG...
4,1104,27577,NZ_KX711708.1_4,TTCACTGAACGTATTTGGCTGGCATTTATACAGGAGCACATGTGCA...
0,1105,105067,NZ_KX774387.1_0,AGCGGGCGGCCGGAAGGTGAATGCTAGGCATGATCTAACCCTCGGT...


In [ ]:
l = 1000
lengthFrame = df.loc[df.apply(lambda row: row[contigLength] > l * fragmentRatio, axis = 1)]
totalLength = np.sum(lengthFrame[contigLength])
numberOfFragments = min(maximumFragments, int(np.ceil(totalLength*coverage/float(l))))
lengthFrame["probability"] = lengthFrame.apply(lambda row: float(row[contigLength])/float(totalLength), axis = 1)

In [ ]:
print(totalLength)
print(numberOfFragments)
lengthFrame

93124453
465623


,contig_id,contig_length,contig_name,contig_sequence,probability
0,0,42856,NZ_CP045500.1,GCCGCCGGAACTGCGCTGGCGCGAGTGGATGGGTCGTGTCGAAGCG...,0.000460
1,1,389300,NZ_CP045499.1,TTGGAGAACACGGGGCTCTTTGAGAGGCGCCAGCCAGGGCCCGTTG...,0.004180
2,2,53938,NZ_CP045498.1,TCGCATGCCGCATGCCGAGCTCGTCATGAGCGCGTTTGAGTTGCGC...,0.000579
3,3,305508,NZ_CP045497.1,ACGTACGGGCCCGCGACGAAATCGCCGACTTGCAATTCGATGCGGA...,0.003281
4,4,2516,NZ_MT560071.1,TCAAGACGATAGTTACCGGATAAGGCGCAGCAGTCGGACTGAACGG...,0.000027
...,...,...,...,...,...
1,1099,2672,NZ_KX452393.1_1,GTAGGCAGTTCGCTCCAAGCTGGACTGTATGCACGAACCCCCCGTT...,0.000029
0,1100,6723,NZ_KX711708.1_0,ATGAGAATGACGACAAATAAGACTTCCCTTTCTCGCTTAACAAAAG...,0.000072
4,1104,27577,NZ_KX711708.1_4,TTCACTGAACGTATTTGGCTGGCATTTATACAGGAGCACATGTGCA...,0.000296
0,1105,105067,NZ_KX774387.1_0,AGCGGGCGGCCGGAAGGTGAATGCTAGGCATGATCTAACCCTCGGT...,0.001128


In [ ]:
lengthSpecificDf = lengthFrame.copy()
lengthFrame = lengthFrame.sample(n=numberOfFragments, replace = True, weights = lengthFrame["probability"])

In [ ]:
sequencesDf1 = lengthFrame.apply(lambda row: getSequence(row[contigSequence], row[contigLength], l), axis = 1) 

In [ ]:
sequencesDf = pd.DataFrame({"sequence" : sequencesDf1})
sequencesDf.reset_index(inplace = True, drop=True)
sequencesDf["sequenceId"] = sequencesDf.index
sequencesDf

,sequence,sequenceId
0,TCGGTTACAGAGCTCCCGGACGGCCTGAGACCTCTCTTCCGGGCTG...,0
1,GGAAAGCCCTGATATGGCTACTGAAGAATGGTTCGGCTATGGCCGT...,1
2,ATAACATCGCAGGAGTTCGACATGCTCAAGAGACAGCTGATTGCGA...,2
3,CGTGATAATGACGGTAAACGCGCTGACATCAGGAAAAAAGATTTAC...,3
4,AACGTTTTTGCGGTGCGAATTTCTTCGATCAGTTTATTGATATCTG...,4
...,...,...
465618,CGCTGACGACGTGTTTGCCTGGTCCGTGCAGACTCAGTTTCCTTCT...,465618
465619,ATGCCTGATATTTATGAACTTGCCCAACAACGCGCTGAAGTCATTA...,465619
465620,ATGAATACCAGTCAGTTGTCACTCGCGAACAATACGAATCAGCTCT...,465620
465621,TTGTTGCGCTCCATCGCATAGATCATGCGCGTGGTGGTCGCCATGT...,465621


In [ ]:
startTime = datetime.now()
lengthFrame.apply(lambda row: getSequence(row[contigSequence], row[contigLength], l), axis = 1)
print(datetime.now() - startTime)

0:00:08.438573


#Unwanted

In [ ]:
# def parse_seqs(inputfile, min_length = 1000):
#     """Return a list of the seqs and a pandas dataframe with metadata."""
#     if verbose:
#       print("Importing sequences")
#     startTime = datetime.now()
#     initialSequences = SeqIO.parse(inputfile, 'fasta')
#     print("Reading Time \t", datetime.now() - startTime)
#     #TODO:// Can create as a numpy array
#     seqs = [rec for rec in initialSequences if countSequencesInFile() and len(rec.seq) >= min_length and re.match(dnaStringPattern, str(rec.seq[0:])) != None] # NOTE:// np.str got 53 seconds, normal methods 41
#     if verbose:
#       print("Imported ", len(seqs), " sequences")
#     contig_info = pd.DataFrame([
#         {'contig_id': i, 'contig_name': rec.id, 'contig_length': len(rec.seq)}
#         for i, rec in enumerate(seqs)
#     ])
#     startTime = datetime.now()
#     cat = SeqIO.to_dict(initialSequences)
#     print(len(cat))
#     print(cat)
#     print("Sequence Count", seqCount)
#     # validSequenceRatio = len(contig_info) / len(SeqIO.to_dict(initialSequences))
#     print("Precentage Time \t", datetime.now() - startTime)
#     #TOCHECK:// Valid row ratio
#     # print(validSequenceRatio)
#     return seqs, contig_info

In [ ]:
# def parse_seqs(inputfile, min_length = 1000):
#     """Return a list of the seqs and a pandas dataframe with metadata."""
#     if verbose:
#       print("Importing sequences")
#     startTime = datetime.now()
#     initialSequences = SeqIO.parse(inputfile, 'fasta')
#     print("Reading Time \t", datetime.now() - startTime)
#     #TODO:// Can create as a numpy array
#     seqs = [rec for rec in initialSequences] # NOTE:// np.str got 53 seconds, normal methods 41
#     print(datetime.now() - startTime)
#     startTime = datetime.now()
#     if verbose:
#       print("Imported ", len(seqs), " sequences")
#     contig_info = pd.DataFrame([
#         {'contig_id': i, 'contig_name': rec.id, 'contig_length': len(rec.seq)}
#         for i, rec in enumerate(seqs)
#     ])
#     print(datetime.now() - startTime)
#     print("initial dataframe\n")
#     print(contig_info)
#     startTime = datetime.now()
#     print("Total length sequences ", len(contig_info))
#     # seq_tensor = torch.from_numpy(contig_info.to_numpy().astype(str))
#     filtered_contig_info = contig_info.loc[contig_info.apply(lambda row: row['contig_length'] > minLength, axis = 1)]
#     print(filtered_contig_info)
#     print(len(filtered_contig_info))
#     print(datetime.now() - startTime)
#     print(filtered_contig_info['contig_length'])
#     return seqs, contig_info